In [ ]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v nov1122har               # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_ap

In [ ]:
%%time
%%capture
%reset -f
!pip -q install -U plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px
import tensorflow as tf, tensorflow.keras as keras
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from keras.layers import Flatten, Dense
from sklearn.model_selection import train_test_split
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('max_colwidth', 1000, 'max_columns', 100, 'display.width', 1000, 'max_rows', 4)
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 5.45 s, sys: 1.25 s, total: 6.7 s
Wall time: 1min


In [21]:
%time vX  = pd.read_csv('testX.csv', index_col='id')  # load testing input features X (only)
%time tYX = pd.read_csv('trainYX.csv')                # partially load training labels Y and input features X
tYX  # 561 input features

CPU times: user 335 ms, sys: 56.1 ms, total: 391 ms
Wall time: 394 ms
CPU times: user 25.3 s, sys: 6.28 s, total: 31.6 s
Wall time: 32.3 s


,y,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560
0,5,0.2778,0.0092,-0.0676,-0.9785,-0.9160,-0.9610,-0.9834,-0.9170,-0.9590,-0.9390,-0.4230,-0.7520,0.8496,0.6226,0.8400,-0.9434,-0.9614,-1.0370,-1.0150,-1.0070,-0.9640,-0.9550,-0.6772,0.0568,0.0192,0.5900,-0.3162,0.1833,0.4440,-0.2622,0.1092,0.4468,-0.4443,-0.1484,0.1718,-0.2727,0.0954,-0.4720,-0.5264,0.2332,0.9640,-0.1309,0.1071,-0.9814,-0.9480,-0.9727,-0.9720,-0.9575,-0.9585,...,-0.9126,-0.2037,-0.5300,-0.8164,-0.9170,-0.8850,-0.9033,-0.9120,-0.9750,-0.9326,-1.0140,-0.9560,-0.6780,-0.9966,-0.6180,-0.1021,-0.5977,-0.9546,-0.9110,-0.9260,-0.9297,-1.0170,-0.9460,-1.022,-0.9570,-0.2930,-1.0100,-0.3455,-0.1411,-0.5215,-0.9585,-0.9160,-0.9434,-0.9414,-0.9750,-0.9414,-0.9890,-0.9610,-0.4453,-1.002,-0.5415,-0.0308,-0.5093,0.0380,-0.0912,-0.1415,-0.1316,-0.8200,0.1721,-0.0535
1,1,0.2454,0.0073,-0.1046,-0.2010,0.1426,-0.2668,-0.2776,0.0648,-0.2605,-0.0572,-0.0364,-0.2830,-0.2830,-0.1448,0.4443,-0.0844,-0.6733,-0.7603,-0.7847,-0.4136,-0.3633,-0.1837,0.2830,0.5100,0.0582,-0.2502,0.3079,-0.1384,0.0822,0.0902,-0.0034,0.1969,0.0538,0.2996,-0.0258,0.0936,-0.3472,-0.1434,-0.4058,0.3690,0.9326,-0.2942,-0.0916,-0.9966,-0.9640,-0.9663,-0.9746,-0.9736,-0.9634,...,-0.8115,0.4165,-0.4731,-0.8210,0.2542,0.2410,0.2688,0.0928,-0.7710,0.2430,-0.2210,-0.1018,0.7134,-0.8994,-0.0642,-0.0842,-0.4750,-0.1345,-0.3853,-0.2573,-0.5430,-0.7570,-0.1365,-0.677,-0.1826,0.6777,-0.7866,0.3240,-0.6206,-0.8530,-0.2500,-0.3025,-0.3176,-0.3198,-0.6426,-0.2488,-0.7236,-0.2512,0.6177,-0.910,0.1069,-0.0397,-0.4220,0.5480,0.6455,0.2296,-0.0335,-0.7000,0.2998,0.0880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226673,6,0.2737,-0.0115,-0.1252,-0.9766,-0.9927,-0.9660,-1.0030,-0.9473,-0.9326,-0.9233,-0.5350,-0.8060,0.8550,0.6567,0.8223,-0.9750,-0.9946,-0.9750,-1.0060,-0.9834,-0.9460,-0.9634,-0.4097,-0.3337,-0.6070,0.1490,-0.0722,0.0406,0.1503,-0.0634,0.0202,0.1213,0.0300,-0.0611,0.1346,0.0150,-0.3865,0.5190,-0.0378,-0.6133,-0.2202,0.7793,0.6245,-0.9750,-1.0200,-0.9746,-0.9897,-0.9760,-0.9710,...,-0.9834,-0.0245,-0.1919,-0.4775,-0.9375,-0.9736,-0.9720,-0.9880,-1.0205,-0.9840,-0.9946,-1.0050,-0.8887,-1.0330,-0.0442,-0.0434,-0.4700,-1.0220,-0.9460,-0.9326,-0.9746,-0.9673,-1.0050,-0.991,-0.9624,-0.5283,-0.9824,-0.0812,-0.1959,-0.5645,-0.9590,-0.9960,-0.9920,-0.9880,-0.9697,-0.9720,-0.9863,-0.9800,-0.7280,-0.984,-0.0157,-0.1059,-0.5430,0.0603,-0.0623,0.3090,0.2150,0.4028,-0.5522,-0.4355
226674,2,0.3160,-0.0366,-0.0669,-0.2770,-0.1725,-0.2343,-0.3430,-0.1821,-0.1731,-0.0073,-0.2270,-0.3335,0.3062,0.1385,0.5700,-0.1846,-0.7030,-0.8726,-0.7310,-0.5190,-0.3923,-0.0086,0.2270,0.3428,0.1090,-0.2620,-0.0791,0.3618,-0.0624,-0.1311,0.0282,0.1847,0.0669,-0.6440,0.5700,-0.4983,0.1154,-0.4644,-0.0525,0.2094,0.8970,-0.3755,0.1177,-1.0160,-0.9663,-0.9575,-1.0090,-0.9688,-0.9260,...,-0.5670,-0.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
tYX

,y,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560
0,5,0.2778,0.0092,-0.0676,-0.9785,-0.9160,-0.9610,-0.9834,-0.9170,-0.9590,-0.9390,-0.4230,-0.7520,0.8496,0.6226,0.8400,-0.9434,-0.9614,-1.0370,-1.0150,-1.0070,-0.9640,-0.9550,-0.6772,0.0568,0.0192,0.5900,-0.3162,0.1833,0.4440,-0.2622,0.1092,0.4468,-0.4443,-0.1484,0.1718,-0.2727,0.0954,-0.4720,-0.5264,0.2332,0.9640,-0.1309,0.1071,-0.9814,-0.9480,-0.9727,-0.9720,-0.9575,-0.9585,...,-0.9126,-0.2037,-0.5300,-0.8164,-0.9170,-0.8850,-0.9033,-0.9120,-0.9750,-0.9326,-1.0140,-0.9560,-0.6780,-0.9966,-0.6180,-0.1021,-0.5977,-0.9546,-0.9110,-0.9260,-0.9297,-1.0170,-0.9460,-1.022,-0.9570,-0.2930,-1.0100,-0.3455,-0.1411,-0.5215,-0.9585,-0.9160,-0.9434,-0.9414,-0.9750,-0.9414,-0.9890,-0.9610,-0.4453,-1.002,-0.5415,-0.0308,-0.5093,0.0380,-0.0912,-0.1415,-0.1316,-0.8200,0.1721,-0.0535
1,1,0.2454,0.0073,-0.1046,-0.2010,0.1426,-0.2668,-0.2776,0.0648,-0.2605,-0.0572,-0.0364,-0.2830,-0.2830,-0.1448,0.4443,-0.0844,-0.6733,-0.7603,-0.7847,-0.4136,-0.3633,-0.1837,0.2830,0.5100,0.0582,-0.2502,0.3079,-0.1384,0.0822,0.0902,-0.0034,0.1969,0.0538,0.2996,-0.0258,0.0936,-0.3472,-0.1434,-0.4058,0.3690,0.9326,-0.2942,-0.0916,-0.9966,-0.9640,-0.9663,-0.9746,-0.9736,-0.9634,...,-0.8115,0.4165,-0.4731,-0.8210,0.2542,0.2410,0.2688,0.0928,-0.7710,0.2430,-0.2210,-0.1018,0.7134,-0.8994,-0.0642,-0.0842,-0.4750,-0.1345,-0.3853,-0.2573,-0.5430,-0.7570,-0.1365,-0.677,-0.1826,0.6777,-0.7866,0.3240,-0.6206,-0.8530,-0.2500,-0.3025,-0.3176,-0.3198,-0.6426,-0.2488,-0.7236,-0.2512,0.6177,-0.910,0.1069,-0.0397,-0.4220,0.5480,0.6455,0.2296,-0.0335,-0.7000,0.2998,0.0880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226673,6,0.2737,-0.0115,-0.1252,-0.9766,-0.9927,-0.9660,-1.0030,-0.9473,-0.9326,-0.9233,-0.5350,-0.8060,0.8550,0.6567,0.8223,-0.9750,-0.9946,-0.9750,-1.0060,-0.9834,-0.9460,-0.9634,-0.4097,-0.3337,-0.6070,0.1490,-0.0722,0.0406,0.1503,-0.0634,0.0202,0.1213,0.0300,-0.0611,0.1346,0.0150,-0.3865,0.5190,-0.0378,-0.6133,-0.2202,0.7793,0.6245,-0.9750,-1.0200,-0.9746,-0.9897,-0.9760,-0.9710,...,-0.9834,-0.0245,-0.1919,-0.4775,-0.9375,-0.9736,-0.9720,-0.9880,-1.0205,-0.9840,-0.9946,-1.0050,-0.8887,-1.0330,-0.0442,-0.0434,-0.4700,-1.0220,-0.9460,-0.9326,-0.9746,-0.9673,-1.0050,-0.991,-0.9624,-0.5283,-0.9824,-0.0812,-0.1959,-0.5645,-0.9590,-0.9960,-0.9920,-0.9880,-0.9697,-0.9720,-0.9863,-0.9800,-0.7280,-0.984,-0.0157,-0.1059,-0.5430,0.0603,-0.0623,0.3090,0.2150,0.4028,-0.5522,-0.4355
226674,2,0.3160,-0.0366,-0.0669,-0.2770,-0.1725,-0.2343,-0.3430,-0.1821,-0.1731,-0.0073,-0.2270,-0.3335,0.3062,0.1385,0.5700,-0.1846,-0.7030,-0.8726,-0.7310,-0.5190,-0.3923,-0.0086,0.2270,0.3428,0.1090,-0.2620,-0.0791,0.3618,-0.0624,-0.1311,0.0282,0.1847,0.0669,-0.6440,0.5700,-0.4983,0.1154,-0.4644,-0.0525,0.2094,0.8970,-0.3755,0.1177,-1.0160,-0.9663,-0.9575,-1.0090,-0.9688,-0.9260,...,-0.5670,-0.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tYX.y.value_counts(sort=False).to_frame().T  # counts of observations in each label category

,5,1,3,4,2,6
y,42784,37961,30224,39267,33073,43366


In [12]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

Students: Keep all your definitions, code, documentation **between** ⏳ symbols.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

In [13]:
!pip install tensor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.9 MB/s 
     |████████████████████████████████| 3.1 MB 10.1 MB/s 
     |████████████████████████████████| 57 kB 3.0 MB/s 
     |████████████████████████████████| 296 kB 35.5 MB/s 
     |████████████████████████████████| 80 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 25.8 MB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
     |████████████████████████████████| 254 kB 51.1 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
  Created wheel for tensor: filename=tensor-0.3.6-py3-none-any.whl size=71569 sha256=39ff313fc62ed4f5747acb85881237ce85bf98e4f28212eaebc745073118af00
  Stored in directory: /root/.cache/pip/wheels/66/49/9d/4c43e89065768fec7ab631d87d9ac21d619b6b0b3d45de1714
  Created wheel for construct: filename=construct-2.10.68-py3-none-any.whl size=59226 sha256=2d3c26d039f55cad8fadfb94ef9631eb06cc906

In [14]:
import tensorflow as tf

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
tX, tY = tYX.drop('y', axis=1).head(50000), tYX.head(50000).y-1               # shift labels by -1 to range {0,1,2,3,4,5}

In [17]:
std = StandardScaler().fit(tX, tY)
tX = std.transform(tX)
# vX = std.transform(vX)

In [18]:
tX = tf.convert_to_tensor(tX, dtype=tf.int64) 
tY = tf.convert_to_tensor(tY, dtype=tf.int64)

In [19]:
tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0) # seed weights and biases

m = keras.models.Sequential([
    Dense(10, kernel_initializer=Init, input_shape=[tX.shape[1]]), # hidden layer with 100 neurons requires 561x100 trainable weights + 100 biases
    Dense(6,  kernel_initializer=Init, activation='softmax')])     # one output neuron for each label. Softmax for multiclass (single label)
m.summary()
loss = tf.keras.losses.SparseCategoricalCrossentropy()              # Maps 0-based integer labels to one-hot encodings
m.compile(loss=loss, optimizer="adam", metrics=["accuracy"])         # Accuracy is not really needed since it's equivalent to loss

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                5620      
                                                                 
 dense_1 (Dense)             (None, 6)                 66        
                                                                 
Total params: 5,686
Trainable params: 5,686
Non-trainable params: 0
_________________________________________________________________


In [23]:
hist = m.fit(tX, tY, epochs=10, validation_split=.3)  # validation loss is decreasing (as it should)

Epoch 1/10
1094/1094 [==============================] - 4s 4ms/step - loss: 0.0708 - accuracy: 0.9708 - val_loss: 0.0784 - val_accuracy: 0.9663
Epoch 2/10
1094/1094 [==============================] - 4s 4ms/step - loss: 0.0701 - accuracy: 0.9723 - val_loss: 0.0775 - val_accuracy: 0.9675
Epoch 3/10
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0712 - accuracy: 0.9707 - val_loss: 0.0748 - val_accuracy: 0.9692
Epoch 4/10
1094/1094 [==============================] - 4s 4ms/step - loss: 0.0689 - accuracy: 0.9721 - val_loss: 0.0745 - val_accuracy: 0.9697
Epoch 5/10
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0695 - accuracy: 0.9719 - val_loss: 0.0752 - val_accuracy: 0.9690
Epoch 6/10
1094/1094 [==============================] - 4s 3ms/step - loss: 0.0689 - accuracy: 0.9717 - val_loss: 0.0756 - val_accuracy: 0.9687
Epoch 7/10
1094/1094 [==============================] - 4s 4ms/step - loss: 0.0692 - accuracy: 0.9715 - val_loss: 0.1363 - val_accuracy:

In [ ]:
print('History object contains: ' + ', '.join(hist.history.keys()))
dfHist = pd.DataFrame(hist.history)
dfHist['epoch'] = dfHist.index+1
f = px.line(dfHist, x='epoch', y='val_loss', title='', markers=True);
f = f.update_layout(height=200, margin=dict(l=0, r=0, t=0, b=0))
f = f.show();

History object contains: loss, accuracy, val_loss, val_accuracy


In [ ]:
pOneHot = m.predict(vX)   # probabilities for each category. Subjects are rows
YLab = [f'{i}/{s}' for i, s in enumerate('walking walking_upstairs walking_downstairs sitting standing laying'.split())]  # column labels
pd.DataFrame(pOneHot[:3,:], columns=YLab).style.background_gradient(cmap='coolwarm', axis=1)  # display first few predictions

93/93 [==============================] - 0s 2ms/step


,0/walking,1/walking_upstairs,2/walking_downstairs,3/sitting,4/standing,5/laying
0,0.999786,0.000206,0.000008,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [ ]:
pY = pd.DataFrame(np.argmax(pOneHot, axis = 1)+1, columns=['y'])  # predicted labels (from 1 to 6)
pY.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906,2907,2908,2909,2910,2911,2912,2913,2914,2915,2916,2917,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946
y,1,6,6,2,1,6,2,6,1,5,6,2,6,5,1,5,3,2,2,2,2,3,4,2,5,1,5,5,5,5,3,3,2,3,3,2,5,5,6,5,3,5,6,6,6,2,2,2,6,3,...,2,5,4,5,2,3,2,6,1,5,1,5,2,3,4,5,2,2,1,3,2,5,5,2,6,3,6,6,6,3,6,5,3,5,1,4,1,3,6,6,2,1,3,2,4,4,6,3,4,2


In [ ]:
ToCSV(pY, 'HAR_baseline')  # generate a CSV submission file for Kaggle

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
1. ...
1. ...

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 70 sec > 60 sec limit!!!


# 💡**Starter Ideas**

1. Try tuning DNN hyperparameters
1. Training set has 500K observations (2GB), but you really don't need them all. They are all bootstrapped (with noise) from the original sample of 7352 observations. In order to stay within Colab runtime limit (CRTL), you can 
  1. use more observations for a shallow DNN, but risk underfitting due to lower model complexity
  1. use fewer observations for a deeper DNN, but risk overfitting to higher model complexity
1. Check out the original related papers about feature engineering for this dataset
1. Try engineering features with [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) and discarding unimportant features via PCA or alternative technique.
1. Consider KMeans/KMedoid or other clustering methods to identify observations, which represent the original 7352 observations. It might require finding 7352 cluster centroids/medoids.
  1. Fast clustering methods: [FAISS](https://github.com/facebookresearch/faiss) (GPU-enabled)
1. For deep NN, consider dropout, batch normalization
1. Try PCA on transposed matrix to find/eliminate highly correlated observations
1. Try [stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling) to ensure each label is proportionally represented in a subsample